In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import seaborn as sns
import matplotlib.pyplot as plt
sns.set()

In [2]:
df = pd.read_excel('test_sample_medical.xlsx')

In [3]:
df.columns = ['gender', 'birthday', 'service', 'num', 'date', 'diagnosis', 'insured_id', 'case_id', 'service_class']

In [4]:
df['birthday'] = pd.to_datetime(df['birthday'])
df['date'] = pd.to_datetime(df['date'])
df['birthday_year'] = df['birthday'].dt.year

In [5]:
df.head()

,gender,birthday,service,num,date,diagnosis,insured_id,case_id,service_class,birthday_year
0,Ж,1965-09-21,"Прием (осмотр, консультация) врача-невролога п...",1,2018-08-13,G43.0,0,0,1231,1965
1,Ж,1965-09-21,"Прием (осмотр, консультация) врача-невролога, ...",1,2018-06-08,G44.2,0,0,5101,1965
2,Ж,1965-09-21,Эхокардиография с физической нагрузкой,1,2018-07-24,I20.9,0,0,4400,1965
3,Ж,1965-09-21,Холтеровское мониторирование сердечного ритма ...,1,2018-07-17,I20.9,0,0,4100,1965
4,Ж,1965-09-21,Маммография (двух молочных желез ),1,2018-08-14,N64.4,0,0,4210,1965


# Готовим таргет

In [6]:
df[df.index.isin(df['diagnosis'].str.len().to_frame().query('diagnosis > 40').index)]

,gender,birthday,service,num,date,diagnosis,insured_id,case_id,service_class,birthday_year
4158,М,1955-10-31,Прием врача общей практики,1,2018-03-09,D64.9; K29.7; K82.9; E03.9; E61.2; E55; K63.9;...,403,516,1121,1955
4164,М,1955-10-31,Повторный прием врача - общей практики,1,2018-09-24,E16.9; K29.0; K86.9; M85.9; E29.1; I84; N15.9;...,403,517,1121,1955
9142,М,1954-06-16,Прием врача-кардиолога,1,2018-07-30,R07.4; I25.1; I49.9; I10; R42; I70.8; I70.2,802,1077,1201,1954
9143,М,1954-06-16,Холестерин ЛПНП,1,2018-07-30,R07.4; I25.1; I49.9; I10; R42; I70.8; I70.2,802,1077,3200,1954
9144,М,1954-06-16,Холестерин общий,1,2018-07-30,R07.4; I25.1; I49.9; I10; R42; I70.8; I70.2,802,1077,3200,1954
...,...,...,...,...,...,...,...,...,...,...
161653,Ж,1970-08-25,Консультация врача гинеколога первичная (сбор ...,1,2018-10-22,"N 93.9, N 85.0-?, ...",18415,21875,6125,1970
161654,Ж,1970-08-25,Микроскопическое (бактериоскопическое) исследо...,1,2018-10-22,"N 93.9, N 85.0-?, ...",18415,21875,3100,1970
161655,Ж,1970-08-25,Взятие материала для микробиологических исслед...,1,2018-10-22,"N 93.9, N 85.0-?, ...",18415,21875,1252,1970
163198,Ж,1996-05-01,"Прием (осмотр, консультация) врача-гастроэнтер...",1,2018-04-07,"K29.6 , K63.9 , K86.1 , K81.1",18720,22195,1211,1996


In [7]:
test = '''К 05.1
К 05.1 К 03.6 К 03.8
К 05.1,К 03.6,К 03.8
К05.1,К03.6,К03.8
К05.1;К03.6;К03.8
N 93.9, N 85.0-?,                               N 72
R07.4; I25.1; I49.9; I10; R42; I70.8; I70.2
K29.6 , K63.9 , K86.1 , K81.1
D64.9; K29.7; K82.9; E03.9; E61.2; E55; K63.9; E10-E14; G47.9; K76.9; K57; K31.9; K86.9; K81.9; K80.
R07.4; I25.1; I49.9; I10; R42; I70.8; I70.2
К04.9 (1.6)                         К04.9 (1.7)
K04.7  Периапикальный абсцесс без полости'
I 11.9, I 67.9, N 20-?,                            К 80
M54.2,D50.9,E14.9,K80.1
К 05.1  К 03.6  К 03.8
К02.1 35
(J06.9)
№87.2
E03.9, -'''

test = test.split('\n')

In [8]:
def get_codes(s):
    pattern = re.compile('([A-ZА-Я]\s*[0-9]+\.*[0-9]*)')
    return pattern.findall(s)

In [9]:
import re

for t in test:
    matches = get_codes(t)
    print(f'"{t}": {matches}')

"К 05.1": ['К 05.1']
"К 05.1 К 03.6 К 03.8": ['К 05.1', 'К 03.6', 'К 03.8']
"К 05.1,К 03.6,К 03.8": ['К 05.1', 'К 03.6', 'К 03.8']
"К05.1,К03.6,К03.8": ['К05.1', 'К03.6', 'К03.8']
"К05.1;К03.6;К03.8": ['К05.1', 'К03.6', 'К03.8']
"N 93.9, N 85.0-?,                               N 72": ['N 93.9', 'N 85.0', 'N 72']
"R07.4; I25.1; I49.9; I10; R42; I70.8; I70.2": ['R07.4', 'I25.1', 'I49.9', 'I10', 'R42', 'I70.8', 'I70.2']
"K29.6 , K63.9 , K86.1 , K81.1": ['K29.6', 'K63.9', 'K86.1', 'K81.1']
"D64.9; K29.7; K82.9; E03.9; E61.2; E55; K63.9; E10-E14; G47.9; K76.9; K57; K31.9; K86.9; K81.9; K80.": ['D64.9', 'K29.7', 'K82.9', 'E03.9', 'E61.2', 'E55', 'K63.9', 'E10', 'E14', 'G47.9', 'K76.9', 'K57', 'K31.9', 'K86.9', 'K81.9', 'K80.']
"R07.4; I25.1; I49.9; I10; R42; I70.8; I70.2": ['R07.4', 'I25.1', 'I49.9', 'I10', 'R42', 'I70.8', 'I70.2']
"К04.9 (1.6)                         К04.9 (1.7)": ['К04.9', 'К04.9']
"K04.7  Периапикальный абсцесс без полости'": ['K04.7']
"I 11.9, I 67.9, N 20-?,            

In [10]:
def convert_codes(x):
    if pd.isna(x):
        return np.nan
    
    x = x.upper()
    codes = get_codes(x)
    first_letters = [c[0] for c in codes]
    
    if len(first_letters) == 0:
        return np.nan
    
    first_letters = np.unique(first_letters)
    rus = ['А', 'В', 'Е', 'К', 'М', 'Н', 'О']
    eng = ['A', 'B', 'E', 'K', 'M', 'H', 'O']
    tr_table = {ord(r): ord(e) for r, e in zip (rus, eng)}
          
    return [l.translate(tr_table) for l in first_letters]

In [11]:
df['target'] = df['diagnosis'].apply(convert_codes)

In [12]:
df['target'].value_counts(dropna=False)

NaN          74477
[K]          17077
[M]          14114
[N]          13936
[J]          11164
             ...  
[L, A]           1
[K, L, R]        1
[D, E, Z]        1
[I, K, R]        1
[K, S]           1
Name: target, Length: 284, dtype: int64

# Предобработка текста

In [14]:
from pymystem3 import Mystem
import nltk
from string import punctuation

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/aminought/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
m = Mystem()
stopwords = nltk.corpus.stopwords.words('russian')

In [16]:
def transform_text(s):
    tokens = m.lemmatize(s.lower())
    tokens = [t.strip() for t in tokens if t not in stopwords and t != ' ' and t.strip() not in punctuation]
    return ' '.join(tokens)

In [17]:
df['service_lemm'] = df['service'].apply(transform_text).values

In [18]:
df['service_lemm']

0         прием осмотр консультация врач невролог повтор...
1         прием осмотр консультация врач невролог иметь ...
2                       эхокардиография физический нагрузка
3         холтеровский мониторирование сердечный ритм дл...
4                               маммография молочный железо
                                ...                        
165396                             мочевина © согласовывать
165397                      консультация врач офтальмология
165398      rg графия шейный отдел позвоночник 2 пройка ции
165399                         консультация врач неврология
165400    транскраниальный сканирование сосуд головной м...
Name: service_lemm, Length: 165401, dtype: object

# Делим на train и test

In [19]:
from sklearn.model_selection import train_test_split

In [20]:
dataset_without_na = df[~df['target'].isna()]

In [40]:
# здесь должна быть стратификация
_, test_indices = train_test_split(
    dataset_without_na.index, 
    test_size=0.3, 
    shuffle=True, 
    random_state=1
)

In [37]:
test = df.iloc[test_indices]
train = df[~df.index.isin(test_indices)].sample(frac=1)

In [38]:
test.shape, train.shape

((27278, 12), (138123, 12))

In [39]:
train.to_csv('train.csv')
test.to_csv('test.csv')